## SN20 BitAgent

### Setting up ...
- importing standard libraries + bittensor, no special sauce required
- fetching subnet 20
- setting up wallet and validator
- getting top miner
- providing protocol (QnATask)

In [1]:
import asyncio
import requests
import bittensor as bt 
from rich import print as rprint
from typing import Optional,List

# working with subnet 20 / upsilon / BitAgent
subnet = bt.metagraph(netuid=20)

# Wallet and validator setup
WALLET_NAME = "TODO" # TODO
HOTKEY_NAME = "TODO" # TODO
vali_wallet =  bt.wallet(name=WALLET_NAME, hotkey=HOTKEY_NAME)
vali_dendrite = bt.dendrite(wallet=vali_wallet)

# get the TOP miner on the subnet
top_miner_uid = int(subnet.I.argmax())
print("Top Miner UID for Subnet 20: ", top_miner_uid)

# the request protocol
class QnATask(bt.Synapse):
    urls: List[str] = []   # not used right now
    datas: List[dict] = [] # used to pass in relevant context, could be a company knowledge base or a set of wikipedia pages
    prompt: str = ""       # the query / prompt
    response: Optional[dict] = {}
    timeout: Optional[float] = 3.0
    miner_uids: Optional[List[int]] = [top_miner_uid] # put our TOP miner into the network as the miner to query (if empty list, a random list of miners will be selected)

    def toJSON(self):
        return {"prompt": self.prompt, 
                "urls": self.urls, 
                "datas": self.datas, 
                "response": self.response,
                "miner_uids": self.miner_uids,
                "dendrite_process_time": self.dendrite.process_time,
                "dendrite_status_code": self.dendrite.status_code,
                "axon_status_code": self.axon.status_code,}

2024-04-07 17:47:36.518 |       INFO       | You are connecting to finney network with endpoint wss://entrypoint-finney.opentensor.ai:443.
2024-04-07 17:47:36.519 |     WARNING      | We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network.
2024-04-07 17:47:36.519 |     WARNING      | In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it.
2024-04-07 17:47:36.833 |       INFO       | Connected to finney network and wss://entrypoint-finney.opentensor.ai:443.
Top Miner UID for Subnet 20:  197


## Two ways to query SN20 

### First way is to use your registered validator to query directly to the TOP miner
- build a task (QnATask) with a "prompt" and optional "datas"
- query the network
- see response answer (1)
- see top citation (2)
- see full response object (3)

In [2]:
task = QnATask(prompt="hey, what does the meow eat?", 
               datas=[{"source": "source 1", "context": "Some irrelevant context"},
                      {"source": "source 2", "context": "meow is a monkey in the jungle"},
                      {"source": "source 3", "context": "meow prefers to eat grapes and berries, but also eats the occassional banana"},
                      {"source": "source 4", "context": "meow climbs trees for fun"},
                      {"source": "source 5", "context": "meow is afraid of snakes, but loves bunnies"}])

responses = vali_dendrite.query(
    axons=[subnet.axons[top_miner_uid]],
    synapse=task,
    deserialize=False,
    timeout=task.timeout,
)

response = responses[0].response
print("1 - Response showing answer from miner: \n\t", response['response'])
print("2 - Response's topmost relevant citation from miner: \n\t", response['citations'])
print("3 - Full response: \n\t", responses)

1 - Response showing answer from miner: 
	  Meow eats grapes, berries, and occasional bananas.
2 - Response's topmost relevant citation from miner: 
	 [{'context': 'meow prefers to eat grapes and berries, but also eats the occassional banana', 'source': 'source 3'}]
3 - Full response: 
	 [QnATask(timeout=3.0, urls=[], datas=[{'source': 'source 1', 'context': 'Some irrelevant context'}, {'source': 'source 2', 'context': 'meow is a monkey in the jungle'}, {'source': 'source 3', 'context': 'meow prefers to eat grapes and berries, but also eats the occassional banana'}, {'source': 'source 4', 'context': 'meow climbs trees for fun'}, {'source': 'source 5', 'context': 'meow is afraid of snakes, but loves bunnies'}], prompt='hey, what does the meow eat?', response={'response': ' Meow eats grapes, berries, and occasional bananas.', 'citations': [{'context': 'meow prefers to eat grapes and berries, but also eats the occassional banana', 'source': 'source 3'}], 'context': 'meow prefers to eat gr

### Second way is to use your validator (or any wallet) to query one of the subnet validators
- we'll use the same QnATask from above, but we'll change the prompt
- query the network via validator axon
- we are specifying the miner uid for our QnATask to be the TOP miner uid
- see response answer (1)
- see top citation (2)
- see full response object (3)

In [3]:
task.prompt = "What does meow fear the most?"

responses = vali_dendrite.query(
    axons=[subnet.axons[0]],
    synapse=task,
    deserialize=False,
    timeout=task.timeout,
)
response = responses[0].response
print("1 - Response showing answer: \n\t", response['response'])
print("2 - Response's topmost relevant citation: \n\t", response['citations'])
print("3 - Full response: \n\t", responses)

1 - Response showing answer: 
	  Meow fears snakes the most.
2 - Response's topmost relevant citation: 
	 [{'context': 'meow is afraid of snakes, but loves bunnies', 'source': 'source 5'}]
3 - Full response: 
	 [QnATask(timeout=3.0, urls=[], datas=[{'source': 'source 1', 'context': 'Some irrelevant context'}, {'source': 'source 2', 'context': 'meow is a monkey in the jungle'}, {'source': 'source 3', 'context': 'meow prefers to eat grapes and berries, but also eats the occassional banana'}, {'source': 'source 4', 'context': 'meow climbs trees for fun'}, {'source': 'source 5', 'context': 'meow is afraid of snakes, but loves bunnies'}], prompt='What does meow fear the most?', response={'response': ' Meow fears snakes the most.', 'citations': [{'context': 'meow is afraid of snakes, but loves bunnies', 'source': 'source 5'}], 'context': 'meow is afraid of snakes, but loves bunnies'}, miner_uids=[])]


# Demonstration of all task types and scoring / incentives ...
## Generating tasks from the Task API
We'll get tasks from the Task API that are far more complicated than the one above - 
- Summarization Task
- QnA with Citations Task
- QnA Logic Task

### Setting up ...
- methods to 
  - get the top miner, 
  - fetch a task 
  - get miner response and 
  - evaluate a task
- setup task IDs for QnA with Citations, Pet Tricks and Summarization

In [11]:
qna_task = (3,1)
pet_tricks_task = (6,6)
summarization_task = (8,1)

def get_top_miner_uid(subnet):
    return subnet.I.argmax()

def get_task(task_id, sub_task_id):
    # keep trying in case it's being restarted 
    while True:
        try:
            resp = requests.post("https://roguetensor.com/api/task_api/get_new_task", json={"task_id": task_id, "sub_task_id": sub_task_id}).json()
            return resp
        except:
            pass

def eval_task(task_id, response):
    # keep trying in case it's being restarted 
    while True:
        try:
            resp = requests.post("https://roguetensor.com/api/task_api/evaluate_task_response", json={"task_id": task_id, "response": response.toJSON()}).json()
            return resp
        except:
            pass

def get_miner_response_to_task(subnet, validator, miner_uid, task):
    print("Fetching response from TOP miner: ", miner_uid)

    response = None
    while not response:
        response = asyncio.run(validator.call(
            # Send the query to selected miner axons in the network.
            target_axon=subnet.axons[miner_uid],
            # Construct a query. 
            synapse=task,
            # All responses have the deserialize function called on them before returning.
            # You are encouraged to define your own deserialization function.
            deserialize=True,
            timeout=25.0
        ))
        return response

def evaluate_miner(subnet, validator, miner_uid, task_id, sub_task_id):
    task_json = get_task(task_id, sub_task_id)
    gen_task_id = task_json["task"]["task_id"]
    task = QnATask(prompt=task_json['task']['prompt'], datas=task_json['task']['datas'], urls=task_json['task']['urls'])
    print("Got task with prompt: ", task_json['task']['prompt'][:60] + " ...")
    miner_response = get_miner_response_to_task(subnet, validator, miner_uid, task)
    print("Miner response:", miner_response.response['response'][:100] + " ...")
    eval = eval_task(gen_task_id, miner_response)
    return miner_response, *eval["result"]

### Generated Summary Task Example:

In [12]:
task_id, sub_task_id = summarization_task
miner_response, score, max_score, results, correct_answer_optional = evaluate_miner(subnet, vali_dendrite, top_miner_uid, task_id, sub_task_id)
print("Scores: ", score, max_score)

Got task with prompt:  Summarize this and make sure to be concise:  Did you attend  ...
Fetching response from TOP miner:  197
Miner response:  Pamela and Marie both did not attend the independence march. Pamela stayed at home due to concerns  ...
Scores:  2.25 2.25


#### The results the miner would see:

In [13]:
rprint(("\n").join(results)) 

Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 0.8873927593231201.
You received 0.5 of 0.5 reward.
Return summary shorter than original
✅ You responded with a valid summary length.
You received 0.5 of 0.5 reward.
Return valid summary
✅ You responded with a valid summary.
You received 1.0 of 1.0 reward.

### Generated QnA with Citations Task Example:

In [14]:
task_id, sub_task_id = qna_task
miner_response, score, max_score, results, correct_answer_optional = evaluate_miner(subnet, vali_dendrite, top_miner_uid, task_id, sub_task_id)
print("Scores: ", score, max_score)

Got task with prompt:   "What was the purpose of integrating Crashlytics into Fireb ...
Fetching response from TOP miner:  197
Miner response:  The purpose of integrating Crashlytics into Firebase was to bring the best of both platforms togeth ...
Scores:  5.25 5.25


#### The results the miner would see:

In [15]:
rprint(("\n").join(results)) 

Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 0.8485991954803467.
You received 0.5 of 0.5 reward.
Returns expected citation source(s)
✅ You correctly identified some or all of the correct citation sources (1/1 identified).
You received 1.5 of 1.5 reward.
Returns a relevant response
✅ You responded with a relevant response compared to the context.
You received 1.0 of 1.0 reward.
Returns a unique response
✅ You responded with a novel response compared to the context.
You received 1.0 of 1.0 reward.
Returns valid response
✅ You responded with a valid response.
You received 1.0 of 1.0 reward.

### Generated Pet Tricks QnA Logic Task Example:

In [16]:
task_id, sub_task_id = pet_tricks_task
miner_response, score, max_score, results, correct_answer_optional = evaluate_miner(subnet, vali_dendrite, top_miner_uid, task_id, sub_task_id)
print("Scores: ", score, max_score)

Got task with prompt:  Given the following Trick Descriptions with numerical IDs:
  ...
Fetching response from TOP miner:  197
Miner response: 4 ...
Scores:  1.75 1.75


#### The results the miner would see:

In [17]:
rprint(("\n").join(results)) 

Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 0.3985426425933838.
You received 0.5 of 0.5 reward.
Returns expected value
✅ You responded with a valid answer.
You received 1.0 of 1.0 reward.

## That's It!

You saw how to:
 1) Query the top miner uid
 2) Demonstrate each reward/penalty mechanism/Scoring of the top miner response
 3) Query the subnet 2 different ways (as a validator to a miner and through a registered validator axon)